# pip & import

In [1]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import category_encoders as ce

In [3]:
apply_train_df = pd.read_csv('apply_train.csv')
resume_train_df = pd.read_csv('resume.csv')
certificate_df = pd.read_csv('resume_certificate.csv')
education_train_df = pd.read_csv('resume_education.csv')
language_train_df = pd.read_csv('resume_language.csv')
company_train_df = pd.read_csv('company.csv')
recruitment_train_df = pd.read_csv('recruitment.csv')

# 함수 정의 & 글로벌 변수

In [4]:
def recall5(answer_df, submission_df):

    primary_col = answer_df.columns[0]
    secondary_col = answer_df.columns[1]

    # submission의 예측이 각각 5개인지 확인
    prediction_counts = submission_df.groupby(primary_col).size()
    if not all(prediction_counts == 5):
        raise ValueError(f"Each {primary_col} should have exactly 5 {secondary_col} predictions.")


    # submission의 예측된 값들에 null값이 있는지 확인
    if submission_df[secondary_col].isnull().any():
        raise ValueError(f"Predicted {secondary_col} contains NULL values.")

    # 예측값에 중복이 있는지 확인
    duplicated_preds = submission_df.groupby(primary_col).apply(lambda x: x[secondary_col].duplicated().any())
    if duplicated_preds.any():
        raise ValueError(f"Predicted {secondary_col} contains duplicates for some {primary_col}.")


    # Filter the submission dataframe based on the primary_col present in the answer dataframe
    # primary_col 즉 resume_seq가 양측에 있는지 확인 후 남김
    submission_df = submission_df[submission_df[primary_col].isin(answer_df[primary_col])]

    # For each primary_col, get the top 5 predicted secondary_col values
    #
    top_5_preds = submission_df.groupby(primary_col).apply(lambda x: x[secondary_col].head(5).tolist()).to_dict()

    # Convert the answer_df to a dictionary for easier lookup
    true_dict = answer_df.groupby(primary_col).apply(lambda x: x[secondary_col].tolist()).to_dict()


    individual_recalls = []
    for key, val in true_dict.items():
        if key in top_5_preds:
            correct_matches = len(set(true_dict[key]) & set(top_5_preds[key]))
            individual_recall = correct_matches / min(len(val), 5) # 공정한 평가를 가능하게 위하여 분모(k)를 'min(len(val), 5)' 로 설정함
            individual_recalls.append(individual_recall)


    recall = np.mean(individual_recalls)
    return recall

In [5]:
def minimum_condition(resume, company):
  if(resume['degree'] < company['education']):
    return False
  else:
    return True
   # if(resume['career_month']>company['career_end'] or resume['career_month']<company['career_start']): 다 0임
     # return False
   # else:


In [6]:
def check_salary(salary):
  if salary>3000:
    return 3
  elif salary>2000:
    return 2
  elif salary>1000:
    return 1
  else:
    return 0

In [7]:
def check_career_month(career_month):
  if career_month>120:
    return 3
  elif career_month>60:
    return 2
  elif career_month>0:
    return 1
  else:
    return 0

In [8]:
def check_univ_score(univ_score):
  if univ_score>=90:
    return 90
  elif univ_score>=80:
    return 80
  elif univ_score>=70:
    return 70
  elif univ_score>=60:
    return 60
  elif univ_score>=50:
    return 50
  elif univ_score>=40:
    return 40
  elif univ_score>=30:
    return 30
  elif univ_score>=20:
    return 20
  elif univ_score>=10:
    return 10
  else:
    return 0

In [48]:
a = 0
b = 0.3
c = 0.01 # 0.0014
d = 0.41

# apply_matrix

In [10]:
#학습, 검증 분리
train, test = [], []
apply_train_groupby = apply_train_df.groupby('resume_seq')['recruitment_seq'].apply(list)
for uid, iids in zip(apply_train_groupby.index.tolist(), apply_train_groupby.values.tolist()):
    for iid in iids[:-1]:
        train.append([uid,iid])
    test.append([uid, iids[-1]])

In [11]:
apply_train = pd.DataFrame(train, columns=['resume_seq', 'recruitment_seq'])
apply_test = pd.DataFrame(test, columns=['resume_seq', 'recruitment_seq'])
apply_pred = apply_train_df.copy()

In [12]:
# 사용자-아이템 행렬 생성: 구직자가 해당 채용 공고에 지원했으면 1, 아니면 0으로 설정
apply_train_item_matrix = apply_train.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
apply_train_similarity = cosine_similarity(apply_train_item_matrix) # 사용자 간의 유사성 계산

apply_test_item_matrix = apply_test.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
apply_test_similarity = cosine_similarity(apply_test_item_matrix) # 사용자 간의 유사성 계산

apply_pred_item_matrix = apply_pred.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
apply_pred_similarity = cosine_similarity(apply_pred_item_matrix) # 사용자 간의 유사성 계산

In [13]:
apply_train_matrix = pd.DataFrame(apply_train_similarity) # DataFrame 생성
apply_test_matrix = pd.DataFrame(apply_test_similarity) # DataFrame 생성
apply_pred_matrix = pd.DataFrame(apply_pred_similarity) # DataFrame 생성

# 행과 열 이름 설정
apply_train_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
apply_train_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

apply_test_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
apply_test_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

apply_pred_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
apply_pred_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

# resume_matrix

## resume_matrix_career_job_code null 없애기

In [14]:
resume_train = resume_train_df.copy()
resume_train = resume_train[['resume_seq', 'degree', 'job_code_seq1', 'updated_date','hope_salary', 'last_salary', 'career_month','text_keyword', 'career_job_code']] # 여기 수정

In [15]:
import warnings
warnings.filterwarnings("ignore")

# 'career_job_code'가 NaN이면서 'text_keyword'에 '상품기획'이 있는 행 선택
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('상품기획'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('MD'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('스타일디자이너'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('일러스트레이터'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('영업기획'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('골프웨어디자이너'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('컬러리스트자격증'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('머천다이저'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('패션'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '디자인;CAD;설계'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('피팅'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '디자인;CAD;설계'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('화학'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '화학;고무;플라스틱'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('캐주얼'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('영업'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('디자이너'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('디자인'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '디자인;CAD;설계'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('마케팅'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('경영'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('여성'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('남성'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('구매'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('인사'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('디스플레이'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '반도체;LCD;광학;정밀'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('사무'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('생산'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('슈퍼바이저'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '신문;서적;출판;인쇄'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('중국'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('일러스트'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('일본어'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('팀'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('삽마스터'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('총무'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('매장관리'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '도소매;유통;무역'
selected_rows = resume_train_df[(resume_train_df['career_job_code'].isnull()) & (resume_train_df['text_keyword'].str.contains('성실한인재'))]
resume_train.loc[selected_rows.index, 'career_job_code'] = '섬유;봉제;가방;의류'
resume_train.loc[resume_train['career_job_code'].isnull(), 'career_job_code'] = '섬유;봉제;가방;의류'

## resume_text

In [16]:
resume_train_text = resume_train_df[['resume_seq', 'text_keyword']]

In [17]:
resume_train_text.fillna('무응답', inplace=True)

In [18]:
# ';'로 구분된 값을 분리하여 새로운 행 생성
new_rows = []
for index, row in resume_train_text.iterrows():
    keywords = row['text_keyword'].split(';')
    for keyword in keywords:
        new_rows.append({'resume_seq': row['resume_seq'], 'text_keyword': keyword})

# 새로운 행을 데이터프레임에 추가
resume_train_text = pd.DataFrame(new_rows)

## resume_matrix 계산

In [19]:
aaa = resume_train_df.copy() # 1에 너무 양이 몰려서 2에서 가져오는건데 데이터를 변경하는거라 좋지 않음
aaa['job_code_seq1'].value_counts() # 희망 직무가 같은게 7288인걸 보면 흐으으으으으으음
aaa.loc[(aaa['job_code_seq1'] == '재료·화학·섬유·의복') & (aaa['job_code_seq2'] != ''), 'job_code_seq1'] = aaa.loc[(aaa['job_code_seq1'] == '재료·화학·섬유·의복') & (aaa['job_code_seq2'] != ''), 'job_code_seq2']
aaa['job_code_seq1'].fillna('재료·화학·섬유·의복', inplace=True)
resume_train['job_code_seq1'] = aaa['job_code_seq1']

In [20]:
resume_train['month'] = resume_train['updated_date'].apply(lambda x : int(x[5:7]))
resume_train['day'] = resume_train['updated_date'].apply(lambda x : int(x[8:10]))

In [21]:
resume_train['sub_career_month'] = resume_train['career_month'].map(lambda x: check_career_month(x))
resume_train['sub_last_salary'] = resume_train['last_salary'].map(lambda x: check_salary(x))
resume_train['sub_hope_salary'] = resume_train['hope_salary'].map(lambda x: check_salary(x))

resume_train = resume_train.drop(columns=['updated_date'])
resume_train = resume_train.drop(columns=['career_month'])
resume_train = resume_train.drop(columns=['last_salary'])
resume_train = resume_train.drop(columns=['hope_salary'])
resume_train = resume_train.drop(columns=['text_keyword'])

In [22]:
resume_train['degree'] = resume_train['degree'].astype("category") # 원핫 인코딩 할 것들(숫자)
resume_train['sub_career_month'] = resume_train['sub_career_month'].astype("category")
resume_train['sub_last_salary'] = resume_train['sub_last_salary'].astype("category")
resume_train['sub_hope_salary'] = resume_train['sub_hope_salary'].astype("category")

In [23]:
_train_encoded = pd.DataFrame()

for col in ['degree', 'sub_career_month', 'sub_last_salary','sub_hope_salary']: # 수정
  _encoder = ce.OneHotEncoder(use_cat_names=True)
  _encoder.fit(resume_train[col])
  _encoded = _encoder.transform(resume_train[col])
  _train_encoded = pd.concat([_train_encoded, _encoded], axis=1)

In [24]:
resume_tmp = resume_train[['resume_seq']]
resume_tmp = pd.merge(resume_tmp, _train_encoded, left_index=True, right_index=True, how='inner')

In [25]:
resume_item_matrix = resume_train.groupby(['resume_seq', 'job_code_seq1']).size().unstack(fill_value=0)
resume_item_matrix2 = resume_train.groupby(['resume_seq', 'career_job_code']).size().unstack(fill_value=0)
resume_text_matrix = resume_train_text.groupby(['resume_seq', 'text_keyword']).size().unstack(fill_value=0)

resume_item_matrix = pd.merge(resume_item_matrix, resume_tmp, on='resume_seq', how='left')
resume_item_matrix = pd.merge(resume_item_matrix, resume_item_matrix2, on='resume_seq', how='left')
resume_item_matrix = pd.merge(resume_item_matrix, resume_text_matrix, on='resume_seq', how='left')

resume_item_matrix.set_index('resume_seq', inplace=True)
resume_similarity = cosine_similarity(resume_item_matrix) # 사용자 간의 유사성 계산

In [26]:
resume_matrix = pd.DataFrame(resume_similarity) # DataFrame 생성
# 행과 열 이름 설정
resume_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
resume_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

# certificate

In [27]:
certificate_train = certificate_df.copy()
tmp = apply_train_df.copy()
tmp = tmp.groupby('resume_seq')['recruitment_seq'].apply(';'.join).reset_index()
tmp = tmp.drop(columns = ['recruitment_seq'])

In [28]:
# NaN 값을 빈 문자열로 대체하고, 문자열로 형변환
certificate_train['certificate_contents'] = certificate_train['certificate_contents'].fillna('').astype(str)

# 자동차 운전면허와 관련된 내용을 필터링
certificate_train = certificate_train[~certificate_train['certificate_contents'].str.contains('자동차 운전면허')]
certificate_train = certificate_train[~certificate_train['certificate_contents'].str.contains('운전')]
certificate_train = certificate_train[~certificate_train['certificate_contents'].str.contains('면허')]
'''certificate_train = certificate_train[~certificate_train['certificate_contents'].str.contains('워드')]
certificate_train = certificate_train[~certificate_train['certificate_contents'].str.contains('유통')]
certificate_train = certificate_train[~certificate_train['certificate_contents'].str.contains('샵')]
certificate_train = certificate_train[~certificate_train['certificate_contents'].str.contains('활')]'''
# 1인 건을 필터링하여 새로운 데이터프레임 생성
certificate_train = certificate_train[certificate_train.groupby('certificate_contents')['certificate_contents'].transform('count') > 1]
certificate_train = certificate_train[certificate_train.groupby('certificate_contents')['certificate_contents'].transform('count') < 200]
# 필터링된 결과를 확인
print(certificate_train['certificate_contents'].value_counts())

컬러리스트 산업기사     167
샵마스터 3급        141
컬러리스트 기사       134
GTQ 포토샵 1급     125
패션머천다이징산업기사    114
              ... 
품질관리기사           2
GTQ 일러스트 3급      2
메이크업             2
6시그마 GB          2
레크레이션1급          2
Name: certificate_contents, Length: 699, dtype: int64


In [29]:
certificate_item_matrix = certificate_train.groupby(['resume_seq', 'certificate_contents']).size().unstack(fill_value=0)
certificate_item_matrix = pd.merge(tmp, certificate_item_matrix, on='resume_seq', how='left')
certificate_item_matrix.fillna(0, inplace=True)
certificate_item_matrix.set_index('resume_seq', inplace=True)
certificate_similarity = cosine_similarity(certificate_item_matrix) # 사용자 간의 유사성 계산

In [30]:
certificate_matrix = pd.DataFrame(certificate_similarity) # DataFrame 생성
# 행과 열 이름 설정
certificate_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
certificate_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

# education_matrix

In [31]:
education_train = education_train_df.copy()
education_train = education_train_df[['resume_seq', 'hischool_special_type', 'hischool_type_seq',  'hischool_gender', 'univ_type_seq1','univ_major_type', 'univ_score']]

In [32]:
education_train['sub_univ_score'] = education_train['univ_score'].map(lambda x: check_univ_score(x))

education_train = education_train.drop(columns=['univ_score'])

In [33]:
education_train['hischool_type_seq'] = education_train['hischool_type_seq'].astype("category")# 원핫 인코딩 할 것들(숫자)
education_train['hischool_gender'] = education_train['hischool_gender'].astype("category")
education_train['univ_type_seq1'] = education_train['univ_type_seq1'].astype("category")
education_train['univ_major_type'] = education_train['univ_major_type'].astype("category")
education_train['sub_univ_score'] = education_train['sub_univ_score'].astype("category")

In [34]:
_train_encoded = pd.DataFrame()

for col in ['hischool_type_seq',  'hischool_gender', 'univ_type_seq1','univ_major_type', 'sub_univ_score']: # 수정
  _encoder = ce.OneHotEncoder(use_cat_names=True)
  _encoder.fit(education_train[col])
  _encoded = _encoder.transform(education_train[col])
  _train_encoded = pd.concat([_train_encoded, _encoded], axis=1)

In [35]:
education_tmp = education_train[['resume_seq']]
education_tmp = pd.merge(education_tmp, _train_encoded, left_index=True, right_index=True, how='inner')

In [36]:
education_item_matrix = education_train.groupby(['resume_seq', 'hischool_special_type']).size().unstack(fill_value=0)
education_item_matrix = pd.merge(education_item_matrix, education_tmp, on='resume_seq', how='left')

In [37]:
education_item_matrix.set_index('resume_seq', inplace=True)
education_similarity = cosine_similarity(education_item_matrix) # 사용자 간의 유사성 계산

# language

# 회사 최소 조건

In [38]:
resume_c = resume_train_df.copy()
recruitment_c = recruitment_train_df.copy()

In [39]:
# 'resume_seq' 열을 기준으로 정렬
resume_c = resume_c.sort_values(by='resume_seq')
# 인덱스를 순차적으로 변경
resume_c['resume_seq'] = [f'U{i:05}' for i in range(1, len(resume_c) + 1)]
resume_c = resume_c.set_index('resume_seq')

# 'resume_seq' 열을 기준으로 정렬
recruitment_c = recruitment_c.sort_values(by='recruitment_seq')
# 인덱스를 순차적으로 변경
recruitment_c['recruitment_seq'] = [f'R{i:05}' for i in range(1, len(recruitment_c) + 1)]
recruitment_c = recruitment_c.set_index('recruitment_seq')

In [40]:
resume_c = resume_c[['degree']]
recruitment_c = recruitment_c[['education']]

In [41]:
apply_item_matrix_new = []
count = 0
# 초등학생들 ^^
tem_6 = [] # U05886이 6학년
tem_5 = [] # U08422이 5학년
tem_4 = [] # U04600이 4학년
tem_3 = [] # U03400이 3학년
tem_2 = [] # U04220이 2학년

for recruitment_seq in range(len(recruitment_c)): #
  if resume_c.loc['U05886']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        a = False
  else:
        a = True
  tem_6.append(a)

for recruitment_seq in range(len(recruitment_c)):
  if resume_c.loc['U08422']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        a = False
  else:
        a = True
  tem_5.append(a)

for recruitment_seq in range(len(recruitment_c)):
  if resume_c.loc['U04600']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        a = False
  else:
        a = True
  tem_4.append(a)

for recruitment_seq in range(len(recruitment_c)):
  if resume_c.loc['U03400']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        a = False
  else:
        a = True
  tem_3.append(a)

for recruitment_seq in range(len(recruitment_c)):
  if resume_c.loc['U04220']['degree'] < recruitment_c.iloc[recruitment_seq]['education']:
        a = False
  else:
        a = True
  tem_2.append(a)

# apply_item_matrix의 인덱스(=resume_seq)와 컬럼(=recruitment_seq)을 뽑아냄
for resume_seq in range(len(resume_c)):
    if resume_c.iloc[resume_seq]['degree'] == 6:
      apply_item_matrix_new.append(tem_6)
    elif resume_c.iloc[resume_seq]['degree'] == 5:
      apply_item_matrix_new.append(tem_5)
    elif resume_c.iloc[resume_seq]['degree'] == 4:
      apply_item_matrix_new.append(tem_4)
    elif resume_c.iloc[resume_seq]['degree'] == 3:
      apply_item_matrix_new.append(tem_3)
    else:
      apply_item_matrix_new.append(tem_2)

# 새로운 DataFrame 생성
apply_item_matrix_new = pd.DataFrame(apply_item_matrix_new, index=apply_pred_item_matrix.index, columns=apply_pred_item_matrix.columns)
apply_item_matrix_new

recruitment_seq,R00001,R00002,R00003,R00004,R00005,R00006,R00007,R00008,R00009,R00010,...,R06686,R06687,R06688,R06689,R06690,R06691,R06692,R06693,R06694,R06695
resume_seq,,,,,,,,,,,,,,,,,,,,,
U00001,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U00002,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U00003,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U00004,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U00005,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
U08478,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
U08479,True,True,True,True,True,False,True,True,True,True,...,True,True,True,True,True,True,True,False,False,True
U08480,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


# train 결과 도출

In [49]:
# train_사용자 간의 유사성 계산
user_train_similarity = a*apply_train_similarity + b*resume_similarity + c*certificate_similarity + d*education_similarity

In [50]:
user_matrix = pd.DataFrame(user_train_similarity) # DataFrame 생성
# 행과 열 이름 설정
user_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
user_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

In [51]:
sort_user_list = []
applied_jobs = []
for idx, user in enumerate(user_matrix.index):
    # 해당 사용자가 지원한 채용 공고
    applied_job = apply_train[apply_train['resume_seq'] == user]['recruitment_seq'].values
    applied_jobs.append([user, applied_job])

    # 해당 사용자의 최종 유사도 (높은 유사도부터 정렬)
    sorted_user_indices = user_train_similarity[idx].argsort()[::-1]
    sort_user_list.append([user,sorted_user_indices])

In [52]:
# 이미 지원한 채용 공고 제외하고 추천
recommendations = []
count = 0

for idx, user in enumerate(sort_user_list):
  re_applied_jobs = []
  for idxx, i in enumerate(user[1]):
    for idxxx, job in enumerate(applied_jobs[i][1]):
      if(len(re_applied_jobs) >= 5):
          break
      else:
        if ((job not in applied_jobs[idx][1]) and (apply_item_matrix_new.loc[user[0]].loc[job]) and (job not in re_applied_jobs)):
          re_applied_jobs.append(job)
          recommendations.append([user[0], job])

    if(len(re_applied_jobs)>=5):
      break

In [53]:
val_prediction = pd.DataFrame(recommendations, columns=['resume_seq', 'recruitment_seq'])

In [54]:
recall5(apply_test,val_prediction)

0.003890591841546805

# pred 결과 도출

In [ ]:
# 사용자 간의 유사성 계산
user_similarity = a*apply_pred_similarity + b*resume_similarity+ c*certificate_similarity

In [ ]:
user_matrix = pd.DataFrame(user_similarity) # DataFrame 생성
# 행과 열 이름 설정
user_matrix.columns = [f'U{i:05d}' for i in range(1, 8483)]
user_matrix.index = [f'U{i:05d}' for i in range(1, 8483)]

In [ ]:
sort_user_list = []
applied_jobs = []
for idx, user in enumerate(user_matrix.index):
    # 해당 사용자가 지원한 채용 공고
    applied_job = apply_train[apply_train['resume_seq'] == user]['recruitment_seq'].values
    applied_jobs.append([user, applied_job])

    # 해당 사용자의 최종 유사도 (높은 유사도부터 정렬)
    sorted_user_indices = user_similarity[idx].argsort()[::-1]
    sort_user_list.append([user,sorted_user_indices])

In [ ]:
# 이미 지원한 채용 공고 제외하고 추천
recommendations = []
count = 0

for idx, user in enumerate(sort_user_list):
  re_applied_jobs = []
  for idxx, i in enumerate(user[1]):
    for idxxx, job in enumerate(applied_jobs[i][1]):
      if(len(re_applied_jobs) >= 5):
          break
      else:
        if ((job not in applied_jobs[idx][1]) and (apply_item_matrix_new.loc[user[0]].loc[job]) and (job not in re_applied_jobs)):
          #and (resume_c.loc[user[0]]['degree'] < recruitment_c.loc[job]['education'])
          re_applied_jobs.append(job)
          recommendations.append([user[0], job])

    if(len(re_applied_jobs)>=5):
      break

In [ ]:
# sample_submission.csv 형태로 DataFrame 생성
top_recommendations = pd.DataFrame(recommendations, columns=['resume_seq', 'recruitment_seq'])

top_recommendations.to_csv('./try_submit.csv', index=False)